## Hayes Valley, San Francisco - Where is best to park based on the street sweeping schedule?

#### This notebook accesses the city of San Francisco's street cleaning data and prepares the Hayes Valley (HV) parking permit streets to be displayed on a simple web map. The goal of the web map is to display where one can park with a HV parking permit for the longest amount of time before the next street cleaning. This notebook prepares a feature class to be published to AGOL that includes up to 4 different street sweeping times per road segment. This process could be reiterated for any other parking permit zone of San Francisco. 

In [ ]:
#load libraries 
from arcgis.features import GeoAccessor
import pandas as pd
import arcpy, os, math, requests
from io import StringIO

In [ ]:
gdb = r'C:\Users\ordem\Documents\ArcGIS Projects\Playground\Playground.gdb' #set the location of project gdb

#### Load the San Francisco street sweeping data into a pandas dataframe and remove any duplicate rows. 

In [ ]:
# access the SF street sweeping data 
# https://data.sfgov.org/City-Infrastructure/Street-Sweeping-Schedule/yhqp-riqs/about_data
url = "https://data.sfgov.org/resource/yhqp-riqs.csv" # socrata API endpoint for the dataset

In [ ]:
params = {
    "$limit": 40000,  # there are 37.5K entries, so a 40K limit is sufficient
    "$offset": 0      # starting point for pagination
}

response = requests.get(url, params=params) # use requests library to pull data from the url 
df = pd.read_csv(StringIO(response.text)) # read the response string into a pandas dataframe        

In [ ]:
duplicate_counts = df.duplicated().sum() #get count of duplicate rows 
print ('Number of duplicate rows:', duplicate_counts) 
df_removeDuplicate = df.drop_duplicates(keep='last') # remove those duplicate rows and keep just one copy 

#### Create a unique identifier for each street segment. This is a number (CNN) combined with R (right side) or L (left side). 

In [ ]:
def addUniqueID (row): #function to add a column (cnn_RL) for street ID plus right or left 
    row['cnn_RL'] = str(row['cnn']) + str(row['cnnrightleft'])
    return row

In [ ]:
df_ID = df_removeDuplicate.apply (addUniqueID, axis = 1) #apply the addUniqueID function to each row of the dataframe 

#### Apply a pivot to get a single row for each unique street segment and create columns to hold up to 4 different street sweeping times.

In [ ]:
df_forPivot = df_ID[[ 'cnn_RL', 'weekday', 'fromhour', 'tohour']].copy() #make a copy of the dataframe with only the relevant columns of the pivot 
df_forPivot['cumcount'] = df_forPivot.groupby('cnn_RL').cumcount() #calculate a column with the identifier number (0,1,2,3...) of each entry in each cnn_RL group 
group_sizes = df_forPivot.groupby('cnn_RL').transform('size')  #get the size of each cnn_Rl group 
df_forPivot['flag'] = (group_sizes > 4).astype(int) # add a flag for any groups with more than 4 entires (i.e. more than 4 street sweeping times per road segment) 
df_forPivot = df_forPivot.sort_values(by='cnn_RL') # sort dataframe by cnn_RL

In [ ]:
df_filtered = df_forPivot[df_forPivot['flag'] == 0] #This removes all road segments with more than 4 street sweeping times for simplicity (removes flag ==1)
df_pivot = df_filtered.pivot_table(index= 'cnn_RL', columns = ['cumcount'], values=['weekday', 'fromhour', 'tohour'], aggfunc = 'first') #pivot the table using the cumulative count field to have unique identifiers for multiple fromhour, tohour, and weekday fields

In [ ]:
df_pivot.columns = [f"{col[0]}_{col[1]}" for col in df_pivot.columns] #rename the columns to flatten the column IDs (i.e. fromhour_0)
df_pivot.reset_index(inplace=True)

In [ ]:
df_meta = df_ID[[ 'cnn_RL', 'cnn', 'corridor', 'limits', 'cnnrightleft', 'line']].copy() #create a copy of the original dataframe and copy just the relevant metadata fields 
df_result = df_pivot.merge(df_meta, on = 'cnn_RL', how = 'left') # join with the pivoted dataframe 
df_combined = df_result.drop_duplicates(subset=['cnn_RL']) # drop any duplicates that resulted from the join 

df_combined #view the combined dataframe 

#### Convert the dataframe to a feature class saved in a local geodatabase.

In [ ]:
sdf = GeoAccessor.from_df(df_combined, geometry_column='line') #create a spatial dataframe using the geometry of each line 
sdf.spatial.to_featureclass(os.path.join(gdb, 'SF_Street_Sweeping_Pivoted')) #convert the sdf to a feature class saved in the project geodatabase

#### Add additional parking data to the map to query only the streets where there is free time limited parking and a HV permit exception.

In [ ]:
feature_service_url = "https://services.sfmta.com/arcgis/rest/services/Parking/parkingregulations_timelimited/FeatureServer/0" #add the San Francisco time limited parking layer from web 
aprx = arcpy.mp.ArcGISProject('CURRENT') # reference the current ArcGIS Pro project
m = aprx.listMaps()[0] #get the first map
data = m.addDataFromPath(feature_service_url) 
arcpy.management.MakeFeatureLayer(data, data.name) # add the time limited parking feature service as a layer to the map 

In [ ]:
layer = m.listLayers('Time Limits w/RPP exceptions')[0] #reference the layer just added 
layer.definitionQuery = "RPPAREA1 = 'HV' Or RPPAREA2 = 'HV'" #apply a definition query on Time Limited Parking for only Hayes Valley (HV) permit streets 
#this provides only the streets where there is time limited free parking (no meters) and there is an HV permit exception 

In [ ]:
arcpy.analysis.Buffer(
    in_features= 'Time Limits w/RPP exceptions',
    out_feature_class=r"memory\TimeLimitedParking_Hayes_Buffer",
    buffer_distance_or_field="30 Feet",
    line_side="FULL",
    line_end_type="ROUND",
    dissolve_option="ALL",
    dissolve_field=None,
    method="PLANAR"
) # create a 30 ft buffer around the time limited parking (HV only) streets in order to select streets within this zone 

In [ ]:
arcpy.management.SelectLayerByLocation(
    in_layer="SF_Street_Sweeping_Pivoted",
    overlap_type="HAVE_THEIR_CENTER_IN",
    select_features=r"memory\TimeLimitedParking_Hayes_Buffer",
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
) #select by location the street sweeping data within 30 ft buffer of the time limited parking areas 

In [ ]:
new_fc = 'SF_Street_Sweeping_Pivoted_HayesValleyPermitArea' #name the new feature class
arcpy.conversion.ExportFeatures(
    in_features="SF_Street_Sweeping_Pivoted",
    out_features= os.path.join(gdb, new_fc),
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping='Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,SF_Street_Sweeping_Pivoted,Shape_Length,-1,-1;cnn_rl "cnn_rl" true true false 13 Text 0 0,First,#,SF_Street_Sweeping_Pivoted,cnn_rl,0,12;fromhour_0 "fromhour_0" true true false 8 BigInteger 0 0,First,#,SF_Street_Sweeping_Pivoted,fromhour_0,-1,-1;fromhour_1 "fromhour_1" true true false 254 Text 0 0,First,#,SF_Street_Sweeping_Pivoted,fromhour_1,0,253;fromhour_2 "fromhour_2" true true false 254 Text 0 0,First,#,SF_Street_Sweeping_Pivoted,fromhour_2,0,253;fromhour_3 "fromhour_3" true true false 254 Text 0 0,First,#,SF_Street_Sweeping_Pivoted,fromhour_3,0,253;tohour_0 "tohour_0" true true false 8 BigInteger 0 0,First,#,SF_Street_Sweeping_Pivoted,tohour_0,-1,-1;tohour_1 "tohour_1" true true false 254 Text 0 0,First,#,SF_Street_Sweeping_Pivoted,tohour_1,0,253;tohour_2 "tohour_2" true true false 254 Text 0 0,First,#,SF_Street_Sweeping_Pivoted,tohour_2,0,253;tohour_3 "tohour_3" true true false 254 Text 0 0,First,#,SF_Street_Sweeping_Pivoted,tohour_3,0,253;weekday_0 "weekday_0" true true false 7 Text 0 0,First,#,SF_Street_Sweeping_Pivoted,weekday_0,0,6;weekday_1 "weekday_1" true true false 7 Text 0 0,First,#,SF_Street_Sweeping_Pivoted,weekday_1,0,6;weekday_2 "weekday_2" true true false 7 Text 0 0,First,#,SF_Street_Sweeping_Pivoted,weekday_2,0,6;weekday_3 "weekday_3" true true false 7 Text 0 0,First,#,SF_Street_Sweeping_Pivoted,weekday_3,0,6;cnn "cnn" true true false 8 BigInteger 0 0,First,#,SF_Street_Sweeping_Pivoted,cnn,-1,-1;corridor "corridor" true true false 41 Text 0 0,First,#,SF_Street_Sweeping_Pivoted,corridor,0,40;limits "limits" true true false 59 Text 0 0,First,#,SF_Street_Sweeping_Pivoted,limits,0,58;cnnrightleft "cnnrightleft" true true false 6 Text 0 0,First,#,SF_Street_Sweeping_Pivoted,cnnrightleft,0,5',
    sort_field=None
) # export the new feature class 

#### Reproject the feature class to use the California State Plane (Zone 3) Coordinate System in order to measure the line segments in feet.

In [ ]:
arcpy.management.Project(
    in_dataset= os.path.join(gdb, new_fc),
    out_dataset= os.path.join(gdb, new_fc + '_reprojectFT'),
    out_coor_system='PROJCS["NAD_1983_2011_StatePlane_California_III_FIPS_0403_Ft_US",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",6561666.666666666],PARAMETER["False_Northing",1640416.666666667],PARAMETER["Central_Meridian",-120.5],PARAMETER["Standard_Parallel_1",37.06666666666667],PARAMETER["Standard_Parallel_2",38.43333333333333],PARAMETER["Latitude_Of_Origin",36.5],UNIT["Foot_US",0.3048006096012192]]',
    transform_method="WGS_1984_(ITRF08)_To_NAD_1983_2011",
    in_coor_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
    preserve_shape="NO_PRESERVE_SHAPE",
    max_deviation=None,
    vertical="NO_VERTICAL"
) # reproject feature class to State Plane California Zone 3

#### Since the right and left sides of the roads currently share the same geometry, copy each line parallel and offset by 15ft either to the left or right. 

In [ ]:
# copy parallel segements to split roads into right and left sides 
# see code reference: https://support.esri.com/en-us/knowledge-base/how-to-create-parallel-features-similar-to-the-copy-par-000024330
infc= os.path.join(gdb, new_fc + '_reprojectFT') #reference the reprojected layer
workspace = os.path.dirname(infc)
edit = arcpy.da.Editor(workspace) #open an edit session
edit.startEditing(False,True)
edit.startOperation()

def CopyParallel(plyP,sLength, side): #function to copy each road segment parallel either right or left by 15ft 
    part=plyP.getPart(0)
    newArray=arcpy.Array()
    for ptX in part:
        dL=plyP.measureOnLine(ptX)
        ptX0=plyP.positionAlongLine (dL-0.01).firstPoint
        ptX1=plyP.positionAlongLine (dL+0.01).firstPoint
        dX=float(ptX1.X)-float(ptX0.X)
        dY=float(ptX1.Y)-float(ptX0.Y)
        lenV=math.hypot(dX,dY)

        if side == "L":
            sX = -dY * sLength / lenV
            sY = dX * sLength / lenV
        elif side == "R":
            sX = dY * sLength / lenV 
            sY = -dX * sLength / lenV
        else:
            return None

        newP = arcpy.Point(ptX.X + sX, ptX.Y + sY)
        newArray.add(newP)
    section=arcpy.Polyline(newArray)
    return section

with arcpy.da.UpdateCursor(infc,("Shape@", "cnnrightleft")) as cursor:
    for shp, side in cursor:
        newLine = CopyParallel(shp, 15, side) # 15 ft parallel distance 
        if newLine:
            cursor.updateRow((newLine, side))

del cursor
edit.stopOperation()
edit.stopEditing(True) #stop editing 

#### Since moving the lines creates some overlap at the line ends, trim the lines by 30ft at each end to clean up the layer.

In [ ]:
# reduce lines by 30ft at each end to prevent overlap where lines were moved 
# see code reference: https://ianbroad.com/arcgis-toolbox-shorten-polylines-percentage-distance-arcpy/
def trim_polylines_copy(input_fc, output_fc, trim_length): #function to trim lines by 30ft 
    if arcpy.Exists(output_fc):
        arcpy.Delete_management(output_fc) # ensure the output feature class does not already exist

    arcpy.CopyFeatures_management(input_fc, output_fc) # copy the input feature class to create a working copy

    spatial_ref = arcpy.Describe(input_fc).spatialReference 
    print (spatial_ref.name) # print spatial reference of the input feature class

    with arcpy.da.UpdateCursor(output_fc, ["SHAPE@"]) as cursor: # trim the lines in the copied feature class
        for row in cursor:
            polyline = row[0]
            total_length = polyline.getLength('GEODESIC', 'Feet')
            if total_length > 2 * trim_length: # ensure the polyline is long enough to trim
                new_polyline = polyline.segmentAlongLine(trim_length, total_length - trim_length)
                row[0] = new_polyline
                cursor.updateRow(row)
            else:
                print(f"Skipping a short polyline (Length: {total_length})")

    print(f"Processing complete. Trimmed feature class saved as: {output_fc}")
output = infc + '_trimmed' 
trim_polylines_copy(infc, output , 30) #run the function and save a copy of the trimmed fc 

#### Lastly, publish this layer to AGOL. Use the current time to update the road segment symbology based on how many hours until the next street sweeping. 
#### See the final product [here](https://emmaordemann.maps.arcgis.com/apps/instant/basic/index.html?appid=ddf351b4c51e4de2ade64d87863b6f42). 